In [4]:
import pandas as pd
import sys,os,inspect
import calendar
from datetime import datetime
from datetime import timedelta

from datetime import date

import pygsheets

import PySimpleGUI as sg
#import PySimpleGUIWeb as sg
import pathlib
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import textwrap
import re
from shlex import split
import webbrowser
import subprocess
import collections
import pyglet,tkinter
from pyglet import font
font.add_file('/etc/fonts/fonts/CENTAUR.TTF')

In [5]:
def exceptionLog(exception,funCall):
  exception_message = str(exception)
  exception_type, exception_object, exception_traceback = sys.exc_info()
  filename = os.path.split(exception_traceback.tb_frame.f_code.co_filename)[1]
  print(f"{exception_message} {exception_type} {funCall}, Line {exception_traceback.tb_lineno}")


def toGoogleSheet(row):
    '''Reads the Inventory google sheet and gets the datasets title and cross-refs it to the Socrata 4x4 id.  Also
    gets the fields by 4x4 dataset id and by the title'''
    # scope = ['https://www.googleapis.com/auth/spreadsheets.readonly',
    #          "https://www.googleapis.com/auth/drive.file",
    #               "https://www.googleapis.com/auth/drive"]

    path='../client_secret.json'
    gc=pygsheets.authorize(service_account_file=path)
    sh=gc.open('Changes/Fixes Requested by BIC')
    wk1=sh[0]
    ret = wk1.append_table(row)
    return ret

def getLogSummary():
    '''Reads the Inventory google sheet and gets the datasets title and cross-refs it to the Socrata 4x4 id.  Also
    gets the fields by 4x4 dataset id and by the title'''
    # scope = ['https://www.googleapis.com/auth/spreadsheets.readonly',
    #          "https://www.googleapis.com/auth/drive.file",
    #               "https://www.googleapis.com/auth/drive"]

    path='../client_secret.json'
    gc=pygsheets.authorize(service_account_file=path)
    sh=gc.open('Changes/Fixes Requested by BIC')
    wk1=sh[0]
    df = wk1.get_as_df()
    x=df.values.tolist()
    
    return x,list(df.columns)

def showLogSummary(values,columns):
    layout = [[sg.Text(f"Log Summary",font="CENTAUR 15 bold")],
              [sg.Button("Quit")],
               [sg.Table(values=values,text_color="black", auto_size_columns=False,enable_events=True,num_rows=10,font="CENTAUR 10",
                   justification='left',vertical_scroll_only=False,background_color="#F8C471",
                   alternating_row_color="#FAE5D3",key='-LOGSUMMARY-',headings = columns)]
               ]
    

#    sg.theme(colorTheme)     
    window2 = sg.Window("Log Summary",layout,finalize=True,resizable=True)
    a = window2.CurrentLocation()
    screen_width, screen_height = window2.get_screen_dimensions()
    win_width, win_height = window2.size
    x, y = (screen_width - win_width)//2, (screen_height - win_height)//2
    x=200
    y=200
    window2.move(x, y)
    tableFile1 = window2['-LOGSUMMARY-']
    tableFile1.bind('<Button-1>', "Click")
   # headerStore[tableFile1] = columns
   # statsStore[tableFile1] = stats1
   
    return window2

def addRecord():
       today = date.today()
       mon = str(today.month).rjust(2,"0")
       day = str(today.day).rjust(2,"0") 
       today = f"{today.year}-{mon}-{day}"
       layout = [
                  [sg.Button("Quit")],
                  [sg.Button("Add")],
                   [sg.Text("    When ",font="CENTAUR 15 bold",justification="right"),
                   sg.Input(today,key="-WHEN-")
                  ],
                   [sg.Text("      Who ",font="CENTAUR 15 bold",justification="right"),
                   sg.Input("Joe Comeaux",key="-WHO-")
                  ],
                   [sg.Text(" Dataset ",font="CENTAUR 15 bold",justification="right"),
                   sg.Input("",key="-DATASET-")
                  ],
                  [sg.Text("      4x4 ",font="CENTAUR 15 bold",justification="right"),
                   sg.Input("",key="-W4x4-")
                  ],
                  [sg.Text("    Title ",font="CENTAUR 15 bold",justification="right"),
                   sg.Input("",key="-TITLE-")
                  ]  ,
                  [sg.Text("Message ",font="CENTAUR 15 bold",justification="right"),
                   sg.Multiline("",key="-MESSAGE-",s=(50,10))
                  ],
                  [sg.Text("    Notes ",font="CENTAUR 15 bold",justification="right"),
                   sg.Input("",key="-NOTES-")
                  ],
                  [sg.Text("      JIRA ",font="CENTAUR 15 bold",justification="right"),
                   sg.Input("" ,key="-JIRA-")
                  ],
                  [sg.Text("       UID ",font="CENTAUR 15 bold",justification="right"),
                   sg.Input(today,key="-UID-")
                  ]  
               
               ]
       window2 = sg.Window("Log Summary",layout,finalize=True,resizable=True)
       a = window2.CurrentLocation()
       screen_width, screen_height = window2.get_screen_dimensions()
       win_width, win_height = window2.size
       x, y = (screen_width - win_width)//2, (screen_height - win_height)//2
    

In [7]:
def fixLog():
    
    vals,head= getLogSummary()
   

    layout = [
               [sg.Button("Close")],
               [sg.Button("Add Record")],
               [sg.Table(values=vals,text_color="black", auto_size_columns=False,enable_events=True,num_rows=10,font="CENTAUR 10",
                   justification='left',vertical_scroll_only=False,background_color="#F8C471",
                   alternating_row_color="#FAE5D3",key='-LOGSUMMARY-',headings = head)]
               ]
    

    window2 = sg.Window("Log Summary",layout,finalize=True,resizable=True)
    a = window2.CurrentLocation()
    screen_width, screen_height = window2.get_screen_dimensions()
    win_width, win_height = window2.size
    x, y = (screen_width - win_width)//2, (screen_height - win_height)//2

     
    try: 
        while True:

         #   event, values = window2.read()
            wid, event, values = sg.read_all_windows()
            print('event: ',event)

            if event == sg.WIN_CLOSED or event == 'Close':
                window2.close()
                break
            elif event ==  event == 'Quit':
                wid.close()
            elif event ==  event == 'Add Record':
                addRecord()  
            elif event == "Add":
             #print("Values ",values) 
                cols = ["-WHEN-","-WHO-","-DATASET-","-W4x4-","-TITLE-",
                        "-MESSAGE-","-NOTES-","-JIRA-","-UID-"]
                row=[]
                for col in cols:
                    row.append(values[col])
                    
                ret = toGoogleSheet(row)
                print("Return :",ret)
                
                print(row)
    except  Exception as err: 
       exceptionLog(err,inspect.currentframe().f_code.co_name)
fixLog()

event:  Close


In [8]:
a = date.today()
a.year

2023

In [11]:
mon = str(a.month).rjust(2,"0")
day = str(a.day).rjust(2,"0")

b = f"{a.year}-{mon}-{a.day}"
b

'2023-09-11'